In [4]:
!pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [1]:
import pandas as pd
from util import fetch_song_chart_by_month, with_spotify_track_metadata


In [2]:
def prepare_song_chart():
    songs_chart_by_month = fetch_song_chart_by_month()
    songs_chart_by_month = with_spotify_track_metadata(songs_chart_by_month)
    songs_chart_by_month.to_parquet("data/raw/song_chart_by_month.pq")
    songs_chart_by_month.to_csv("data/grammy_recordings.csv.gz", compression="gzip")
    return songs_chart_by_month

In [4]:
song_chart_summary_df = (
    pd.read_parquet("data/raw/song_chart_by_month.pq")
        .groupby(["spotify_track_id", "spotify_artist_id", "spotify_album_id"]).aggregate({
        "month": "count",
        "indicativerevenue": "sum",
        "position": ["min", "mean", "median"],
    })
    .reset_index()
    .reset_index()
    .set_axis(["index", "id", "artist_id", "album_id", "months", "indicativerevenue", "position_min", "position_mean", "position_median"], axis=1)
    .drop(columns=["index"])
)

song_chart_summary_df.to_csv("data/song_chart_summary.csv", index=False)
song_chart_summary_df.to_parquet("data/song_chart_summary.pq")
song_chart_summary_df

,id,artist_id,album_id,months,indicativerevenue,position_min,position_mean,position_median
0,003FTlCpBTM4eSqYSWPv4H,3vAaWhdBR38Q02ohXqaNHT,0TvOeelcHQXYgPcyQiLhyR,1,1182.013,47,47.000000,47.0
1,005lwxGU1tms6HGELIcUv9,6jJ0s89eD6GaHleKKya26X,0r2BUyPTmpbfuz4rR39mLl,8,15482.793,1,14.250000,12.5
2,009ImBOrIUlWgla8U05RAC,6jJ0s89eD6GaHleKKya26X,3jB9yFDwRe3KhtGnHXJntk,4,3685.317,12,25.000000,25.5
3,00FROhC5g4iJdax5US8jRr,59wfkuBoNyhDMQGCljbUbA,2dHr0LpUe6CNV5lNsr8x0W,1,1913.610,21,21.000000,21.0
4,00Mb3DuaIH1kjrwOku9CGU,0p4nmQO2msCgU4IF37Wi3j,3zXjR3y2dUWklKmmp6lEhy,4,7754.159,15,18.500000,19.0
...,...,...,...,...,...,...,...,...
3308,7zL9xrz5krsuC6qVNluQQw,4iHNK0tOyZPYnBU7nGAgpQ,43iBTEWECK7hSnE0p6GgNo,5,8440.983,11,26.000000,22.0
3309,7zLV30iLRt2GsEQgUEY2OL,4KkHjCe8ouh8C2P9LPoD4F,57qvObQidfcN8quFMvQOXm,7,7471.540,5,17.428571,13.0
3310,7zMcNqs55Mxer82bvZFkpg,06HL4z0CvFAxyc27GXpf02,7mzrIsaAjnXihW3InKjlC3,4,3225.695,32,35.500000,35.5
3311,7zmqbdEqmQWgU2wOoBPG83,4SQxI8xg6LcSWL3KuGVHLa,4MJcuG31fFG8rimuSCvJyx,1,496.469,46,46.000000,46.0


In [9]:
!pwd

/Users/pez/DataspellProjects/music-mining/notebooks
